"""
RESUMEN DE TRANSFORMACIONES APLICADAS:

1. LEADS:
   - Normalización de nombres de columnas
   - Validación de teléfonos (formato español)
   - Procesamiento robusto de edad con tipo Int64
   - Conversión de formato europeo de números
   - Validación de escalas (urgencia 1-5, lead_score 0-100)
   - Creación de categorías demográficas y de negocio

2. CURSOS:
   - Eliminación de registros inválidos
   - Normalización de texto y espacios
   - Conversión de tipos numéricos
   - Formateo de fechas
   - Cálculo de métricas derivadas (descuento, intensidad)
   - Categorización por precio

3. SEGUIMIENTO:
   - Eliminación de columnas irrelevantes
   - Conversión de fechas y booleanos
   - Limpieza robusta de columnas numéricas
   - Normalización de categorías (Title Case)
   - Eliminación de valores negativos
   - Deduplicación por ID

4. PRIMER CONTACTO:
   - Eliminación de columnas irrelevantes
   - Limpieza completa de texto
   - Procesamiento de fechas y horas (formato SQL)
   - Imputación inteligente de valores faltantes
   - Conversión de tipos optimizados
   - Deduplicación completa

ARCHIVOS GENERADOS:
- leads_limpio.csv
- cursos_limpio.csv  
- seguimiento_limpio.csv
- primer_contacto_limpio.csv

Los datos están ahora listos para análisis exploratorio, visualización
y modelado predictivo.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import gspread
from google.oauth2.service_account import Credentials

# Analizamos los datos del fichero 'Leads'

In [ ]:
df_leads = pd.read_csv('../data/Leads.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')


In [ ]:
df_leads

In [ ]:
df_leads.shape

In [ ]:
df_leads.columns

In [ ]:
df_leads.info()

In [ ]:
#df_leads.columns = df_leads.columns.str.strip()
df_leads.columns = df_leads.columns.str.lower()

In [ ]:
df_leads.columns

In [ ]:
if 'fecha_registro' in df_leads.columns:
    df_leads['fecha_registro'] = pd.to_datetime(df_leads['fecha_registro'])

In [ ]:
if 'telefono' in df_leads.columns:
    df_leads['telefono'] = df_leads['telefono'].astype(str)

In [ ]:
df_leads.loc[df_leads["telefono"].str.len() > 9,"telefono"] = None

In [ ]:
if 'edad' in df_leads.columns:
    df_leads['edad'] = pd.to_numeric(df_leads["edad"], errors= "coerce")
    df_leads['edad'] = df_leads['edad'].fillna(0).astype(int)
    df_leads['edad'] = df_leads['edad'].replace(0, np.nan)
    df_leads["edad"] = df_leads["edad"].astype('Int64')
    

In [ ]:
df_leads['presupuesto_estimado']  

In [ ]:
df_leads ['presupuesto_estimado'] = df_leads ['presupuesto_estimado'].str.replace('.', '').str.replace(',', '.').astype(float)


In [ ]:
df_leads ['presupuesto_estimado']

In [ ]:
df_leads['presupuesto_estimado'] = df_leads['presupuesto_estimado'].fillna(0)
df_leads['presupuesto_estimado'] = df_leads['presupuesto_estimado'].round(2).astype(float)


In [ ]:
df_leads ['presupuesto_estimado']

In [ ]:
if 'urgencia_formacion' in df_leads.columns:
    df_leads['urgencia_formacion'] = pd.to_numeric(df_leads['urgencia_formacion'], errors='coerce')
    df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].apply(lambda x: np.nan if pd.notna(x) and x > 5 else x)
    df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].astype('Int64')
    

In [ ]:
if 'lead_score' in df_leads.columns:
    df_leads['lead_score'] = pd.to_numeric(df_leads['lead_score'], errors='coerce')
    df_leads['lead_score'] = df_leads['lead_score'].apply(lambda x: np.nan if pd.notna(x) and x > 100 else x)
    df_leads['lead_score'] = df_leads['lead_score'].astype('Int64')

In [ ]:
if 'codigo' in df_leads.columns:
    df_leads['codigo'] = df_leads['codigo'].astype(int)

In [ ]:
text_columns = ['nombre', 'apellidos', 'email', 'ciudad', 'provincia', 'nivel_estudios', 'situacion_laboral', 'objetivo_profesional','dispositivo_registro', 'horario_preferido', 'modalidad_preferida',
'comercial_asignado', 'fuente', 'campana']

In [ ]:
for col in text_columns:
    if col in df_leads.columns:
        df_leads[col] = df_leads[col].str.strip()
        if col in ['nombre', 'apellidos', 'ciudad', 'provincia', 'comercial_asignado']:
            df_leads[col] = df_leads[col].str.title()

In [ ]:
df_leads['email'] = df_leads['email'].str.lower()


In [ ]:
if 'lead_id' in df_leads.columns:
    duplicados_id = df_leads['lead_id'].duplicated().sum()

In [ ]:
if 'email' in df_leads.columns:
    duplicados_email = df_leads['email'].duplicated().sum()
    print(f"Duplicados por email: {duplicados_email}")
    if duplicados_email > 0:
        print("Emails duplicados:")
        print(df_leads[df_leads['email'].duplicated(keep=False)]['email'].value_counts())

In [100]:
duplicados_totales = df_leads.duplicated().sum()
if duplicados_totales > 0:
    df_leads = df_leads.drop_duplicates()

In [ ]:
if 'edad' in df_leads.columns:
    df_leads['grupo_edad'] = pd.cut(df_leads['edad'], 
                             bins=[0, 25, 35, 45, 55, 100], 
                             labels=['18-25', '26-35', '36-45', '46-55', '56+'],
                             right=False)

In [ ]:
if 'presupuesto_estimado' in df_leads.columns:
    df_leads['categoria_presupuesto'] = pd.cut(df_leads['presupuesto_estimado'], 
                                        bins=[0, 3000, 5000, 7000, 100000], 
                                        labels=['Bajo', 'Medio', 'Alto', 'Pro'],
                                        right=False)

In [ ]:
if 'lead_score' in df_leads.columns:
    df_leads['calidad_lead'] = pd.cut(df_leads['lead_score'], 
                               bins=[0, 40, 60, 80, 100], 
                               labels=['Bajo', 'Medio', 'Alto', 'Excelente'],
                               right=False)

In [ ]:
df_leads.info()

# Analizamos los datos del fichero 'Cursos'

In [ ]:
df_cursos = pd.read_csv('../data/Cursos.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [ ]:
df_cursos

In [ ]:
df_cursos = df_cursos.dropna(subset=['Curso ID'])
df_cursos = df_cursos.dropna(how='all')

In [ ]:
df_cursos

In [ ]:
text_columns = ['Nombre', 'Modalidad', 'Tecnologias', 'Nivel', 'Requisitos previos', 'Descripción', 'Certificación oficial']
for col in text_columns:
    if col in df_cursos.columns:
        df_cursos[col] = df_cursos[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

In [ ]:
df_cursos.shape

In [ ]:
df_cursos.info()

In [ ]:
numeric_columns = ['Duracion meses', 'Horas totales', 'Precio base', 'Precio descuento', 
                  'Plazas disponibles', 'Rating', 'Empleabilidad']

for col in numeric_columns:
    if col in df_cursos.columns:
        df_cursos[col] = pd.to_numeric(df_cursos[col], errors='coerce')

In [ ]:
if 'Fecha inicio proxima' in df_cursos.columns:
        df_cursos['Fecha inicio proxima'] = pd.to_datetime(df_cursos['Fecha inicio proxima'], errors='coerce')
        df_cursos['Fecha inicio proxima'] = df_cursos['Fecha inicio proxima'].dt.strftime('%Y-%m-%d')


In [ ]:
if 'Precio base' in df_cursos.columns and 'Precio descuento' in df_cursos.columns:
    df_cursos['Descuento_porcentaje'] = ((df_cursos['Precio base'] - df_cursos['Precio descuento']) / df_cursos['Precio base'] * 100).round(2)

In [ ]:
if 'Horas totales' in df_cursos.columns and 'Duracion meses' in df_cursos.columns:
    df_cursos['Horas_por_mes'] = (df_cursos['Horas totales'] / df_cursos['Duracion meses']).round(2)

In [ ]:
if 'Precio base' in df_cursos.columns:
    df_cursos['Categoria_precio'] = pd.cut(df_cursos['Precio base'], 
                                         bins=[0, 2000, 5000, float('inf')], 
                                         labels=['Económico', 'Medio', 'Pro'], 
                                         include_lowest=True)

In [ ]:
df_cursos = df_cursos.reset_index(drop=True)


In [ ]:
df_cursos

# Analizamos los datos del fichero 'Seguimiento'

In [ ]:
df_seguimiento = pd.read_csv('../data/Seguimiento.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [ ]:
df_seguimiento

In [ ]:
df_seguimiento.info()

In [ ]:
null_counts = df_seguimiento.isnull().sum()
null_counts


In [ ]:
columns_to_drop = ['irrelevante_1', 'irrelevante_2', 'irrelevante_3']
df_seguimiento = df_seguimiento.drop(columns=columns_to_drop)

In [ ]:
date_columns = ['fecha_ultima_interaccion', 'fecha_matricula', 'fecha_inicio_curso']
for col in date_columns:
    df_seguimiento[col] = pd.to_datetime(df_seguimiento[col], errors='coerce')

In [ ]:
df_seguimiento.info()

In [ ]:
boolean_columns = ['conversion', 'cambio_curso', 'financiacion']
for col in boolean_columns:
    df_seguimiento[col] = df_seguimiento[col].astype(bool)


In [ ]:
numeric_columns = ['num_interacciones', 'duracion_proceso_dias', 'descuento_aplicado', 'precio_final', 'probabilidad_retorno', 'valoracion_experiencia']
for col in numeric_columns:
    df_seguimiento[col] = df_seguimiento[col].astype(str)
    df_seguimiento[col] = df_seguimiento[col].str.strip()
    df_seguimiento[col] = df_seguimiento[col].replace(['', 'nan', 'null', 'None'], np.nan)
    df_seguimiento[col] = pd.to_numeric(df_seguimiento[col], errors='coerce')


In [ ]:
df_seguimiento['estado_final'] = df_seguimiento['estado_final'].str.strip().str.title()


In [ ]:
print(df_seguimiento['estado_final'].value_counts())


In [ ]:
df_seguimiento['metodo_pago'] = df_seguimiento['metodo_pago'].str.strip().str.title()

In [ ]:
print(df_seguimiento['metodo_pago'].value_counts())

In [ ]:
df_seguimiento['modalidad_final'] = df_seguimiento['modalidad_final'].str.strip().str.title()

In [ ]:
print(df_seguimiento['modalidad_final'].value_counts())

In [ ]:
nombre_columns = ['comercial', 'closer']
for col in nombre_columns:
    df_seguimiento[col] = df_seguimiento[col].str.strip().str.title()

In [ ]:
numeric_columns_for_validation = ['num_interacciones', 'duracion_proceso_dias', 'descuento_aplicado', 'precio_final', 'probabilidad_retorno', 'valoracion_experiencia']
for col in numeric_columns_for_validation:
    negative_count = df_seguimiento[df_seguimiento[col] < 0].shape[0]
    if negative_count > 0:
        df_seguimiento.loc[df_seguimiento[col] < 0, col] = np.nan

In [ ]:
initial_rows = len(df_seguimiento)
df_seguimiento = df_seguimiento.drop_duplicates(subset=['seguimiento_id'], keep='first')
final_rows = len(df_seguimiento)
duplicates_removed = initial_rows - final_rows

# Analizamos los datos del fichero 'Primer contacto'

In [ ]:
df_contacto = pd.read_csv('../data/Primer contacto.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [ ]:
df_contacto.info()

In [ ]:
datos_nulos = df_contacto.isnull().sum()

In [ ]:
porcentaje = (datos_nulos / len(df_contacto)) * 100
porcentaje

In [ ]:
df_contacto = df_contacto.drop(columns=['irrelevante_1', 'irrelevante_2', 'irrelevante_3'])


In [ ]:
text_columns = ['medio_contacto', 'comercial', 'estado_contacto', 'objeciones_principales', 
                'notas_adicionales', 'interes_otros_cursos', 'areas_interes_adicional',
                'preferencia_fecha_inicio', 'preferencia_financiacion', 'disponibilidad_horaria_detallada',
                'conocimiento_competencia', 'referenciado_por', 'proxima_accion']
for col in text_columns:
    df_contacto[col] = df_contacto[col].astype(str).str.strip()
    df_contacto[col] = df_contacto[col].str.replace(r'\s+', ' ', regex=True)
    df_contacto[col] = df_contacto[col].replace(['null', 'undefined', 'nan', 'n/a', 'na', 'none', 'nan'], np.nan)
 


In [ ]:
df_contacto['fecha_contacto'] = pd.to_datetime(df_contacto['fecha_contacto'], errors='coerce')
df_contacto['fecha_proxima_accion'] = pd.to_datetime(df_contacto['fecha_proxima_accion'], errors='coerce')
df_contacto['hora_contacto'] = pd.to_datetime(df_contacto['hora_contacto'], errors='coerce').dt.strftime('%H:%M')


In [ ]:
df_contacto['duracion_contacto_min'] = pd.to_numeric(df_contacto['duracion_contacto_min'], errors='coerce')
df_contacto['nivel_interes'] = pd.to_numeric(df_contacto['nivel_interes'], errors='coerce')
df_contacto['presupuesto_confirmado'] = pd.to_numeric(df_contacto['presupuesto_confirmado'], errors='coerce')

In [ ]:
df_contacto['duracion_contacto_min'].fillna(df_contacto['duracion_contacto_min'].median(), inplace=True)
df_contacto['nivel_interes'].fillna(df_contacto['nivel_interes'].median(), inplace=True)

In [ ]:
columnas_rellenar = ['presupuesto_confirmado', 'medio_contacto', 'comercial', 'estado_contacto', 'objeciones_principales', 
                    'notas_adicionales', 'interes_otros_cursos', 'areas_interes_adicional',
                    'preferencia_fecha_inicio', 'preferencia_financiacion', 'disponibilidad_horaria_detallada',
                    'conocimiento_competencia', 'referenciado_por', 'proxima_accion']

for col in columnas_rellenar:
    df_contacto[col].fillna('No especificado', inplace=True)

In [ ]:
df_contacto = df_contacto.drop_duplicates(keep='first')

In [ ]:
df_contacto['duracion_contacto_min'] = df_contacto['duracion_contacto_min'].astype(int)
df_contacto['nivel_interes'] = df_contacto['nivel_interes'].astype(int)

In [ ]:
df_contacto.info()

In [ ]:
df_contacto

In [ ]:
df_leads.to_csv('../data_clean/leads_limpio.csv', sep=';', index=False)
df_cursos.to_csv('../data_clean/cursos_limpio.csv', sep=';', index=False)
df_seguimiento.to_csv('../data_clean/seguimiento_limpio.csv', sep=';', index=False)
df_contacto.to_csv('../data_clean/primer_contacto_limpio.csv', sep=';', index=False)